# 모델에 필요한 파일 받기

In [1]:
#현재 위치 확인
!pwd

/content


In [2]:
#ACGPN 모델 파일 다운로드
!git clone https://github.com/kairess/ACGPN.git

Cloning into 'ACGPN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 165 (delta 21), reused 19 (delta 18), pack-reused 141
Receiving objects: 100% (165/165), 303.15 KiB | 3.29 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [3]:
#다운로드 받은 파일의 위치로 이동
%cd ACGPN

/content/ACGPN


# dependencies 및 사전 파일 준비

In [4]:
!pip install -U --no-cache-dir gdown --pre -qq
!pip install ninja -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 4.2 MB/s eta 0:00:00


In [5]:
import gdown
import numpy as np
from PIL import Image
import IPython
import os
import sys
import time

from predict_pose import generate_pose_keypoints

In [6]:
#모델을 돌리기 위한 파일 준비
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [7]:
#현재 위치 확인
!pwd

/content/ACGPN


In [8]:
#휴먼 세그멘테이션 모델 파일 다운로드
!git clone https://github.com/levindabhi/Self-Correction-Human-Parsing-for-ACGPN.git
#옷 마스킹 모델 파일 다운로드
!git clone https://github.com/levindabhi/U-2-Net.git

Cloning into 'Self-Correction-Human-Parsing-for-ACGPN'...
remote: Enumerating objects: 769, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 769 (delta 72), reused 62 (delta 62), pack-reused 658
Receiving objects: 100% (769/769), 3.80 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Cloning into 'U-2-Net'...
remote: Enumerating objects: 822, done.
remote: Total 822 (delta 0), reused 0 (delta 0), pack-reused 822
Receiving objects: 100% (822/822), 30.72 MiB | 38.59 MiB/s, done.
Resolving deltas: 100% (379/379), done.


# 사전학습된 모델 다운로드

# 포즈 예측 모델

In [9]:
!gdown 1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko -O pose/pose_iter_440000.caffemodel

Downloading...
From (uriginal): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko
From (redirected): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko&confirm=t&uuid=07038ecb-3bc8-4a0f-85e6-344b9a1440e6
To: /content/ACGPN/pose/pose_iter_440000.caffemodel
100% 209M/209M [00:04<00:00, 44.7MB/s]


# 휴먼 세그멘테이션 모델

In [10]:
gdown.download('https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH', 'lip_final.pth', quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=81d302a2-b8ff-4350-bcc9-a94c5c93bb19
To: /content/ACGPN/lip_final.pth
100%|██████████| 267M/267M [00:06<00:00, 43.7MB/s]


'lip_final.pth'

# U-2-Net 모델 (옷 마스크 추출 모델)

In [11]:
#U-2-Net 모델을 돌릴 때 필요한 파일 준비를 위해 해당 모델 파일로 이동
%cd U-2-Net

#U-2-Net 모델에 필요한 파일 생성
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp

#사전 학습된 U-2-Net 모델 다운로드
!gdown 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth

import u2net_load
import u2net_run

#로드하려면 쿠다 필요 (위치 확인용)
u2net = u2net_load.model(model_name='u2netp')

#U-2-Net 모델을 위한 파일 준비가 끝나면 부모 파일로 이동
%cd ..

/content/ACGPN/U-2-Net
mkdir: cannot create directory ‘saved_models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/ACGPN/U-2-Net/saved_models/u2netp/u2netp.pth
100% 4.68M/4.68M [00:00<00:00, 233MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
From (redirected): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ&confirm=t&uuid=d74d9582-b6a0-4000-a894-7312c19b3585
To: /content/ACGPN/U-2-Net/saved_models/u2net/u2net.pth
100% 176M/176M [00:00<00:00, 258MB/s]
...load U2NEP---4.7 MB
/content/ACGPN


# ACGPN모델 (사진 합성 모델)

In [12]:
#ACGPN 모델에 필요한 파일 생성
!mkdir checkpoints

#사전 학습된 ACGPN 모델 다운로드
gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx', output='checkpoints/ACGPN_checkpoints.zip', quiet=False)

#압축된 모델 파일 풀어서 전에 만든 파일위치에 넣어줌
!unzip checkpoints/ACGPN_checkpoints.zip -d checkpoints

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=3a861a19-1cc4-4322-8a14-bebb220c1fa3
To: /content/ACGPN/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:04<00:00, 113MB/s]


Archive:  checkpoints/ACGPN_checkpoints.zip
   creating: checkpoints/label2city/
  inflating: checkpoints/label2city/latest_net_G.pth  
  inflating: checkpoints/label2city/latest_net_G1.pth  
  inflating: checkpoints/label2city/latest_net_G2.pth  
  inflating: checkpoints/label2city/latest_net_U.pth  
  inflating: checkpoints/label2city/opt.txt  


# 가설 1: 전처리부터 추론까지의 코드는 api안에 넣어준다.

In [13]:
!pip install "fastapi[standard]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.6 MB/s eta 0:00:00


In [14]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse, JSONResponse
import os
import time
from PIL import Image


app = FastAPI()

In [17]:

CLOTH_DIR = "inputs/cloth"
MODEL_DIR = "inputs/img"
PREPROCESS_DIR = "Data_preprocessing"
COLOR_DIR = f"{PREPROCESS_DIR}/test_color"
IMG_DIR = f"{PREPROCESS_DIR}/test_img"
EDGE_DIR = f"{PREPROCESS_DIR}/test_edge"
LABEL_DIR = f"{PREPROCESS_DIR}/test_label"
POSE_DIR = f"{PREPROCESS_DIR}/test_pose"
MODEL_RESTORE_PATH = "lip_final.pth"



@app.post("/preprocess_image/cloth")
async def preprocess_clothimage():
    cloth_dir = 'inputs/cloth'
    assert os.path.isdir(cloth_dir), f"{cloth_dir} is not a directory."

    cloth_name = f'cloth_{int(time.time())}.png'

    cloth_path = os.path.join(cloth_dir, sorted(os.listdir(cloth_dir))[0])

    cloth = Image.open(cloth_path)

    cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

    preprocessed_dir = 'Data_preprocessing/test_edge'
    os.makedirs(preprocessed_dir, exist_ok=True)
    preprocessed_path = os.path.join(preprocessed_dir, cloth_name)
    cloth.save(preprocessed_path)

    u2net_run.infer(u2net, 'Data_preprocessing/test_color', preprocessed_dir)

    return FileResponse(preprocessed_path)


In [18]:
#위에 적어놓음
#from fastapi.responses import JSONResponse



@app.post("/preprocess_image/img")
async def preprocess_modelimage():
    img_dir = 'inputs/img'
    assert os.path.isdir(img_dir), f"{img_dir} is not a directory."

    img_name = f'img_{int(time.time())}.png'

    img_path = os.path.join(img_dir, sorted(os.listdir(img_dir))[0])

    img = Image.open(img_path)

    img = img.resize((192,256), Image.BICUBIC)

    preprocessed_dir = 'Data_preprocessing/test_img'
    os.makedirs(preprocessed_dir, exist_ok=True)
    preprocessed_path = os.path.join(preprocessed_dir, img_name)
    img.save(preprocessed_path)

    seg_output_dir = 'Data_preprocessing/test_label'
    os.makedirs(seg_output_dir, exist_ok=True)
    seg_parser_args = f"--dataset 'lip' --model-restore 'lip_final.pth' --input-dir '{preprocessed_dir}' --output-dir '{seg_output_dir}'"
    os.system(f"python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py {seg_parser_args}")

    pose_output_dir = 'Data_preprocessing/test_pose'
    os.makedirs(pose_output_dir, exist_ok=True)
    pose_path = os.path.join(pose_output_dir, img_name.replace('.png', '_keypoints.json'))
    generate_pose_keypoints(preprocessed_path, pose_path)

    seg_path = os.path.join(seg_output_dir, img_name)
    pose_keypoints = load_pose_keypoints(pose_path)

    combined_output_dir = 'Data_preprocessing/test_combine'
    os.makedirs(combined_output_dir, exist_ok=True)
    combine_images(seg_path, pose_keypoints, combined_output_dir)

    combined_path = os.path.join(combined_output_dir, img_name)
    with open(combined_path, mode='rb') as file:
        return JSONResponse(content=file.read(), media_type="image/png")


In [29]:
@app.post("/predict")
async def predict_image():
     with open('Data_preprocessing/test_pairs.txt', 'w') as f:
          f.write(f'{img_name} {cloth_name}')
          os.system(f"python3 test.py")


     result_path = os.path.join('test', 'final', f'{img_name.split(".")[0]}_final.png')
     with open(result_path, "rb") as f:
        result_image = f.read()

        return result_image 